<a href="https://colab.research.google.com/github/codebybishwa/Attendance-Tracker/blob/main/Attendance_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill
from datetime import datetime, timedelta

In [81]:
import re

# Function to read and process the files
def read_files():

    with open('/content/stud_list.txt', 'r') as file:
        students = [line.strip() for line in file.readlines()]

    with open('/content/python dates.txt', 'r') as file:
        content = file.read()

    # Fixing any class_timing without quotes using regex
    content = re.sub(r'class_timing\s*=\s*([0-9]{2}:[0-9]{2})\s*-\s*([0-9]{2}:[0-9]{2})',
                     r'class_timing = "\1 - \2"', content)

    # Creating a dictionary to store the variables from python dates.txt
    date_vars = {}

    try:
        exec(content, {}, date_vars)
    except SyntaxError as e:
        print(f"Error in executing content: {e}")
        return None, None, None, None

    # Extracting the values from date_vars
    lecture_dates = date_vars.get("classes_taken_dates", [])
    missed_dates = date_vars.get("classes_missed_dates", [])
    exam_dates = date_vars.get("exams_dates", [])
    class_timing = date_vars.get("class_timing", "18:00 - 20:00")  # Default value if not present

    attendance = pd.read_csv('/content/input_attendance.csv', parse_dates=['Timestamp'])

    return students, lecture_dates, attendance, class_timing

students, lecture_dates, attendance, class_timing = read_files()

# Printing results to inspect data
print("Students List (First 5):", students[:5])
print("Lecture Dates:", lecture_dates)
print("Class Timing:", class_timing)
print("Attendance Data (First 5 rows):")
print(attendance.head())

Students List (First 5): ['2201CB05 Ade Balakrishna', '2201CB11 Anvaya Sharma', '2201CB19 Chhavi Bamoriya', '2201CB21 Dharmraj Dhaker', '2201CB25 Harshita Singh']
Lecture Dates: ['06/08/2024', '13/08/2024', '20/08/2024', '27/08/2024', '03/09/2024', '17/09/2024', '01/10/2024']
Class Timing: 18:00 - 20:00
Attendance Data (First 5 rows):
             Timestamp                           Roll
0  06/08/2024 18:18:23           2201MM26 Rudra Goyal
1  06/08/2024 18:18:26  2201CE30 Lakshya Pratap Singh
2  06/08/2024 18:18:26       2201CB19 Chhavi Bamoriya
3  06/08/2024 18:18:28          2201CE31 Mayank Kumar
4  06/08/2024 18:18:31         2201CB35 Nakka Supraja


In [82]:
def clean_class_timing(class_timing):
    """
    Ensures that class_timing is a valid string and handles cases where the double quotes are missing.
    Returns a cleaned class_timing string with the proper format 'HH:MM - HH:MM'.
    """
    # Remove any non-numeric, colon, or dash characters at the ends (like spaces or quotes)
    class_timing = class_timing.strip()

    if " - " not in class_timing:
        raise ValueError(f"Invalid class_timing format: {class_timing}")

    return class_timing

def process_attendance(students, lecture_dates, attendance, class_timing):
    """
    Processes attendance data and generates an attendance record indicating if a student
    was absent (0), partially present (1), or fully present (2).
    """

    class_timing = clean_class_timing(class_timing)

    # Splitting the class timing into start and end times
    start_str, end_str = class_timing.split(" - ")
    start_time = datetime.strptime(start_str, "%H:%M").time()
    end_time = datetime.strptime(end_str, "%H:%M").time()

    attendance['Timestamp'] = pd.to_datetime(attendance['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='raise')

    # Initializing a dictionary to store attendance data for each student and each lecture date
    attendance_record = {student: {date: 0 for date in lecture_dates} for student in students}

    for lecture_date in lecture_dates:

        lecture_start_str = f"{lecture_date} {start_time.strftime('%H:%M:%S')}"
        lecture_end_str = f"{lecture_date} {end_time.strftime('%H:%M:%S')}"

        try:
            lecture_start = datetime.strptime(lecture_start_str, '%d/%m/%Y %H:%M:%S')
            lecture_end = datetime.strptime(lecture_end_str, '%d/%m/%Y %H:%M:%S')
        except ValueError as e:

            print(f"Error parsing date: {lecture_date} - {e}")
            continue

        lecture_attendance = attendance[(attendance['Timestamp'] >= lecture_start) & (attendance['Timestamp'] <= lecture_end)]


        for _, row in lecture_attendance.iterrows():
            student_roll = row['Roll']  # student's roll number
            timestamp = row['Timestamp']  # timestamp of their attendance

            if student_roll in attendance_record:
                # Check if the student attended during the full lecture duration
                if lecture_start <= timestamp <= lecture_end:
                    if attendance_record[student_roll][lecture_date] == 0:  # Initially absent
                        attendance_record[student_roll][lecture_date] = 1  # Mark as partially attended
                    elif attendance_record[student_roll][lecture_date] == 1:  # Already partially marked
                        attendance_record[student_roll][lecture_date] = 2  # Mark as fully attended

    return attendance_record


class_timing_test = '18:00 - 20:00'
students_test = ['S001', 'S002', 'S003']
lecture_dates_test = ['01/10/2024', '02/10/2024']
attendance_test = pd.DataFrame({
    'Roll': ['S001', 'S002', 'S003', 'S001'],
    'Timestamp': ['01/10/2024 18:05:00', '02/10/2024 18:30:00', '01/10/2024 19:00:00', '02/10/2024 19:30:00']
})

attendance_test['Timestamp'] = pd.to_datetime(attendance_test['Timestamp'], format='%d/%m/%Y %H:%M:%S')

attendance_record_test = process_attendance(students_test, lecture_dates_test, attendance_test, class_timing_test)


print(attendance_record_test)

{'S001': {'01/10/2024': 1, '02/10/2024': 1}, 'S002': {'01/10/2024': 0, '02/10/2024': 1}, 'S003': {'01/10/2024': 1, '02/10/2024': 0}}


In [83]:
# Process the attendance and output the intermediate attendance record
attendance_record = process_attendance(students, lecture_dates, attendance, class_timing)

# Output intermediate data for checking
print("\nAttendance Record (First 5 Students):")
for student, record in list(attendance_record.items())[:5]:
    print(f"{student}: {record}")


Attendance Record (First 5 Students):
2201CB05 Ade Balakrishna: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 0, '01/10/2024': 2}
2201CB11 Anvaya Sharma: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 2}
2201CB19 Chhavi Bamoriya: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 2}
2201CB21 Dharmraj Dhaker: {'06/08/2024': 2, '13/08/2024': 2, '20/08/2024': 1, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 1}
2201CB25 Harshita Singh: {'06/08/2024': 1, '13/08/2024': 2, '20/08/2024': 2, '27/08/2024': 2, '03/09/2024': 2, '17/09/2024': 2, '01/10/2024': 2}


In [84]:
import openpyxl
from openpyxl.styles import PatternFill

def generate_excel(attendance_record, lecture_dates):

    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = 'Attendance Record'


    ws.cell(row=1, column=1, value='Roll No.')
    for i, date in enumerate(lecture_dates, start=2):
        ws.cell(row=1, column=i, value=date)


    fill_absent = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")  # Red for absent
    fill_partial = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # Yellow for partial
    fill_full = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")    # Green for full


    for row_idx, (student_roll, attendance_data) in enumerate(attendance_record.items(), start=2):
        ws.cell(row=row_idx, column=1, value=student_roll)  # Roll number

        for col_idx, date in enumerate(lecture_dates, start=2):
            attendance_status = attendance_data[date]
            cell = ws.cell(row=row_idx, column=col_idx, value=attendance_status)

            # Applying color based on attendance status
            if attendance_status == 0:
                cell.fill = fill_absent
            elif attendance_status == 1:
                cell.fill = fill_partial
            elif attendance_status == 2:
                cell.fill = fill_full

    wb.save('output_excel.xlsx')

In [85]:
students, lecture_dates, attendance, class_timing = read_files()

In [86]:
attendance_record = process_attendance(students, lecture_dates, attendance, class_timing)

In [87]:
generate_excel(attendance_record, lecture_dates)